In [15]:
import pandas as pd
import sys
import numpy as np
import os
from collections import defaultdict
import json
from itertools import product
import ray
from itertools import combinations
os.environ['RAY_worker_register_timeout_seconds'] = '60'

In [2]:
def RCS_visualization(seq1, seq2):
    '''
        This function takes in two sequence and print out the first seq
        in 5'-3' direction and second seq in 3'-5' direction to visualize 
        the RCS between them
    '''
    complement_dict = {'A':'T', 'G':'C', 'T':'A', 'C':'G', 'N': 'N'}
    reversed_seq2 = seq2[::-1]
    marker = ''
    for i, j in enumerate(seq1):
        if j == complement_dict[reversed_seq2[i]]:
            marker += '-'
        else:
            marker += '*'
    print("5' " + seq1 + " 3'", "   " + marker + "   ", "3' " + reversed_seq2 + " 5'", sep='\n')

In [3]:
# @ray.remote
# class RCS_finder():
#     '''
#         This is a python class that can find the imperfect reverse complementary sequences (RCS) within
#         a intronic sequence ranging from 50bps to 50kbps; it can also find the imperfect RCS of two
#         flanking introns each ranging from 50bps to 50kbps.

#         input_seq1: the Up flanking intron;
#         input_seq2: the Lower flanking intron;
#         is_flanking_introns: boolean value indicating whether dealing with two flanking introns
#         if is_flanking_introns == False, the input_seq2 will be ignored and only input_seq1 will be used;
#         seed_len: seed length for the initial RCS search
#         seq_fraction_of_spacer: the minimal portion of the seq length between the two paired seed region
#                                 when dealing with one intron (between 0 to 1); set spacer to 0 if
#                                 spacer length is not used for filtering;
#         allowed_seed_mismatch: allowed the number of mismatch in the seed region: default is 1 (i.e., 2 score)
#         allowed_max_mismatch: allowed total number of mismatches in the RCS searching: default is 5 (i.e., 10 score)
#         seq_partion: partion of sequence for kmer distribution calculation: (e.g., 3: 3 by 3 grids)
#     '''

#     def __init__(self, key=None, input_seq1=None, input_seq2=None, is_flanking_introns=None, seed_len=None, \
#                  seq_fraction_of_spacer=None, allowed_seed_mismatch=None, allowed_max_mismatch=None,
#                  seq_partition=None):
#         self.key = key
#         self.input_seq1 = input_seq1
#         self.input_seq2 = input_seq2
#         self.is_flanking_introns = is_flanking_introns
#         self.seed_len = seed_len
#         self.seq_fraction_of_spacer = seq_fraction_of_spacer
#         self.allowed_seed_mismatch = allowed_seed_mismatch
#         self.allowed_max_mismatch = allowed_max_mismatch
#         self.seq_partition = seq_partition
# #         self.num_rcm_kmers = None
# #         self.num_rcm_kmers_per_10000_comp = None
# #         self.joint_rcm_kmer_dist = None
# #         self.valid_subseq_pairs_list = None
# #         self.all_possible_subseqs_dict = None
# #         self.rcm_summary_stats = None
# #         self.interval_combinations = None
# #         self.joint_subseq_pair_dist = None

#     def base_conversion(self, seq):
#         '''
#             convert the input sequence into the numerical representation
#         '''
#         base_mapping = {'A': 1, 'T': -1, 'C': 1j, 'G': -1j, 'N': 0}
#         if len(seq) >= 2:
#             seq_map = np.array([base_mapping[i] for i in seq])
#         else:
#             seq_map = base_mapping[seq]
#         return seq_map

#     def get_sub_seq_score(self, seq):

#         '''
#             This function starts with a input sequence and return
#             the cummulative sum of the subsequence of length min_seq
#         '''
#         # get the augumented input sequence
#         aug_seq = 'N' + seq
#         # convert the augumented sequence into the numerical representation
#         aug_seq_M = self.base_conversion(aug_seq)
#         # get the cumulative sum of the augumented sequence
#         aug_seq_cum_M = aug_seq_M.cumsum()
#         # get the cumulative sum of the original input sequence
#         seq_cum_M = aug_seq_cum_M[1:]
#         # get subsequence score with length=self.seed_len
#         sub_seq_score = seq_cum_M[self.seed_len - 1:] - aug_seq_cum_M[:len(aug_seq) - self.seed_len]
#         return sub_seq_score

#     def get_allowed_window_index(self):
#         '''
#             This function takes in the subseq_scores, and a fraction number (i.e., betwen 0 and 1) and return
#             the allowed pair of window index that can be used for further subseqs verification
#         '''
#         if self.is_flanking_introns:
#             # deal with flanking introns
#             subseq_score1 = self.get_sub_seq_score(self.input_seq1).astype(np.complex64)
#             subseq_score2 = self.get_sub_seq_score(self.input_seq2).astype(np.complex64)

#             # put all possible pair-wise subseq addition score in the matrix
#             # first subseq as the column, second subseq as the row
#             subseq_scores_sum = subseq_score1.reshape((-1, 1)) + subseq_score2.reshape((1, -1))

#             # get the sum of the absolute value of imaginary part and real part
#             subseq_scores_abs_sum_mat = abs(subseq_scores_sum.imag) + abs(subseq_scores_sum.real)

#             # get the index for the pair of subseqs that potentially have at most 1 mismatch (i.e., abs sum <= 2)
#             first_window, second_window = np.where(subseq_scores_abs_sum_mat <= self.allowed_seed_mismatch)

#             return first_window, second_window
#         else:
#             # deal with one intron only, use the input_seq1
#             subseq_scores = self.get_sub_seq_score(self.input_seq1)

#             ### this step is fast
#             subseq_scores = subseq_scores.astype(np.complex64)

#             #### this step takes about 7 seconds
#             # put all possible pair-wise subseq addition score in the matrix
#             subseq_scores_sum = subseq_scores.reshape((-1, 1)) + subseq_scores.reshape((1, -1))

#             #### this step takes about 11 seconds
#             # get the sum of the absolute value of imaginary part and real part
#             subseq_scores_abs_sum_mat = abs(subseq_scores_sum.imag) + abs(subseq_scores_sum.real)

#             #### this step takes about 9 seconds
#             # obtain the lower triangular index including the diagnoal
#             tril_index = np.tril_indices(subseq_scores_abs_sum_mat.shape[0])

#             #### this step takes about 7 seconds
#             # change the lower triangular value to be bigger than 2, thus don't need to filter due to the  matrix symetry
#             subseq_scores_abs_sum_mat[tril_index] = self.allowed_seed_mismatch + 1

#             #### this step takes about 9 seconds
#             # get the index for the pair of subseqs that potentially have at most 1 mismatch (i.e., abs sum <= 2)
#             first_window, second_window = np.where(subseq_scores_abs_sum_mat <= self.allowed_seed_mismatch)

#             #### this step takes about 0.9 seconds
#             # get the window distance for each pair of subseqs
#             window_num_diff = second_window - first_window

#             #### this step takes about 0.7 seconds
#             # get the base pair distance for each pair of subseqs # 10 is the default arm length
#             spacer = window_num_diff - self.seed_len

#             ### this step takes about 0.3 seconds; set fraction_of_spacer at least to 0 to
#             ### avoid the overlapping windows
#             # base pair distance for each pair of subseqs at least half the total seq length
#             allowed_index = (spacer >= len(self.input_seq1) * self.seq_fraction_of_spacer)

#             ### these two steps take about 0.6 seconds
#             # get the allowed window index
#             allowed_first_window = first_window[allowed_index]
#             allowed_second_window = second_window[allowed_index]

#             return allowed_first_window, allowed_second_window

#     def subseq_validity_check(self):

#         '''
#             This function is used to check wether the subseq pairs identified in get_allowed_window_index function
#             is valid or not by calling the check_valid_subseq_pairs function,
#             This function also return the relative location of the valid rcm kmers as well as number of
#             kmers per 10000 comparison
#         '''
#         first_window, second_window = self.get_allowed_window_index()

#         valid_subseq_pairs_list = []

#         complement_dict = {'A': 'T', 'G': 'C', 'T': 'A', 'C': 'G', 'N': 'N'}

#         ### check to see if dealing with one introns or flanking introns
#         if self.is_flanking_introns:
#             input_seq1 = self.input_seq1
#             input_seq2 = self.input_seq2
#         else:
#             input_seq1 = self.input_seq1
#             input_seq2 = self.input_seq1

#         for first_index, second_index in zip(first_window, second_window):

#             # set it for each window pair
#             seed_mismatch_score = 0

#             first_subseq, second_subseq = input_seq1[first_index:self.seed_len + first_index], \
#                                           input_seq2[second_index:self.seed_len + second_index]

#             # check for the two ends first to make sure end is matching
#             if first_subseq[0] == complement_dict[second_subseq[-1]]:
#                 # check the interior subseq
#                 for i in range(1, self.seed_len):
#                     if first_subseq[i] != complement_dict[second_subseq[-i - 1]]:
#                         seed_mismatch_score += 2
#                         # if total mismatch already bigger than 2 abort the further checking
#                         if seed_mismatch_score > self.allowed_seed_mismatch:
#                             break

#                 if seed_mismatch_score <= self.allowed_seed_mismatch:
#                     valid_subseq_pairs_list.append((first_subseq, first_index, second_subseq, \
#                                                     second_index, seed_mismatch_score))

#         ### The following code: extract the rcm kmer location distribution relative
#         ### to the corresponding upper or lower intron

#         ### create interval list for upper and lower introns based on the upper and lower intron length

#         upper_equal_space_list = np.linspace(start=0, stop=len(input_seq1), num=self.seq_partition+1)

#         lower_equal_space_list = np.linspace(start=0, stop=len(input_seq2), num=self.seq_partition+1)

#         upper_interval_list = []
#         lower_interval_list = []

#         for i in range(len(upper_equal_space_list) - 1):
#             upper_interval = (upper_equal_space_list[i], upper_equal_space_list[i + 1])
#             upper_interval_list.append(upper_interval)

#         for i in range(len(lower_equal_space_list) - 1):
#             lower_interval = (lower_equal_space_list[i], lower_equal_space_list[i + 1])
#             lower_interval_list.append(lower_interval)

#         ### get the combination of interval from upper and lower introns 5 x 5 interval in this case
#         ### if num = 6
#         interval_combinations = list(product(upper_interval_list, lower_interval_list))
        
#         ## save this interval_combinations to self.interval_combinations for the distribution of extended subseq pairs
#         self.interval_combinations = interval_combinations
        

#         ### get the upper and lower rcm kmer position pairs
#         upper_lower_rcm_kmer_pos_pairs = []
#         for i in valid_subseq_pairs_list:
#             upper_lower_rcm_kmer_pos_pairs.append((i[1], i[3]))

#         ### calculate the total number of rcm kmer in each combination of intervals
#         ### and store them in num_rcm_kmer_cross_intervals list
#         num_rcm_kmer_cross_intervals = []

#         for interval_comb in self.interval_combinations:
#             upper_interval_pos = interval_comb[0]
#             lower_interval_pos = interval_comb[1]
#             ## check if the upper in upper_interval_pos and lower in lower_interval_pos
#             ## upper or lower + 0.5 * self.seed length was used as the center of rcm kmer

#             num_rcm_kmer_cross_intervals.append(sum([(upper + 0.5 * self.seed_len >= upper_interval_pos[0] and\
#                                                       upper + 0.5 * self.seed_len < upper_interval_pos[1]) and\
#                                                      (lower + 0.5 * self.seed_len >= lower_interval_pos[0] and\
#                                                       lower + 0.5 * self.seed_len < lower_interval_pos[1])\
#                                                      for upper, lower in upper_lower_rcm_kmer_pos_pairs]))
#         # if sum(num_rcm_kmer_cross_intervals) == 0:
#         #     joint_rcm_kmer_dist = np.array(num_rcm_kmer_cross_intervals)
#         # else:
#         #     joint_rcm_kmer_dist = np.array(num_rcm_kmer_cross_intervals) / sum(num_rcm_kmer_cross_intervals) * 100
#         ### return the raw number of rcm in different part of the introns maybe better than the normalized version

#         joint_rcm_kmer_dist = list(np.array(num_rcm_kmer_cross_intervals))

#         ### normalize the number of rcm_kmer by the total number of comparision tested
#         ### This is different for flanking introns and within introns

#         # calculate the number of comparison for each allowed seed length: N1 X N2
#         # note this is different from the length of the allowed window indices:
#         # allowed window indices already filtered by the allowed seed_mismach

#         if self.is_flanking_introns:

#             num_comparison = (len(input_seq1) - self.seed_len + 1) * (len(input_seq2) - self.seed_len + 1)
#         else:
#             num_comparison = 1 / 2 * (len(input_seq1) - self.seed_len + 1) * (len(input_seq2) - self.seed_len)

# #         ## get the number of rcm_kmers per 10000 comparison

#         num_rcm_kmers = len(valid_subseq_pairs_list)
#         num_rcm_kmers_per_10000_comp = (num_rcm_kmers / num_comparison) * 10000
        
#         return self.key, num_rcm_kmers, num_rcm_kmers_per_10000_comp, joint_rcm_kmer_dist
        
# #         self.num_rcm_kmers = num_rcm_kmers
# #         self.num_rcm_kmers_per_10000_comp = num_rcm_kmers_per_10000_comp
# #         self.joint_rcm_kmer_dist = joint_rcm_kmer_dist
# #         self.valid_subseq_pairs_list = valid_subseq_pairs_list


# #         return num_rcm_kmers, num_rcm_kmers_per_10000_comp, joint_rcm_kmer_dist, valid_subseq_pairs_list
# #         return self.key, list(joint_rcm_kmer_dist)

# #     def subseq_extension(self):
    
# #         '''
# #             This function is used to extend the subseq pairs in the valid_subseq_pairs_list 
# #             one base at a time until either run out of seq boundary or exceeds the allowed 
# #             max_mismatch_score; it will return the extended subseq pairs in all_possible_subseqs_dict
# #         '''

# #         # invoke the self.subseq_validity_check and used the return value
        
# #         self.subseq_validity_check()
        
# #         ### inwardly extend the candidate subseq_pairs

# #         all_possible_subseqs_dict = {}
        
# #         complement_dict = {'A':'T', 'G':'C', 'T':'A', 'C':'G', 'N': 'N'}
        
        
# #         ### check to see if dealing with one introns or flanking introns
# #         if self.is_flanking_introns:
# #             input_seq1 = self.input_seq1
# #             input_seq2 = self.input_seq2
# #         else:
# #             input_seq1 = self.input_seq1
# #             input_seq2 = self.input_seq1
            
# #         ### define the self.allowed_max_mismatch based one the square root of the shortest one of the flanking introns X 2
# #         if self.allowed_max_mismatch is None:
# #             self.allowed_max_mismatch = np.sqrt(min(len(input_seq1), len(input_seq2))) * 2
            
# #         for pair_index, valid_subseq_pair in enumerate(self.valid_subseq_pairs_list):

# #             # expand the elements in the valid_subseq_pair
# #             first_subseq, first_index, second_subseq, second_index, mismatch_score = valid_subseq_pair

# #             # initial end index of the first subseq 
# #             first_down_index = first_index + self.seed_len -1

# #             # initial start index of the second subseq 
# #             second_up_index = second_index
            
# #             # the extended_sub_seq_pairs collect all the extended subsequences 
# #             # that are within the max_mismatch_score
# #             extended_sub_seq_pairs = []

# #             # append the subseqs, their current start_index, end_index, and mismatch_score
# #             # remember the python list slicing: end is excluded
# #             extended_sub_seq_pairs.append((first_subseq, first_index, first_down_index + 1,
# #                                            second_subseq, second_up_index, second_index + self.seed_len,
# #                                            mismatch_score))

# #             # two different checking criteria for either flanking introns or internal intron
# #             # check to make sure the indexs are within the boundary for the flanking intron case
            
# #             if self.is_flanking_introns:

# #                 # inward extension
# #                 first_down_index += 1
# #                 second_up_index -= 1

# #                 while (first_down_index <= len(input_seq1) - 1) and (second_up_index >= 0):

# #                     first_down_base = input_seq1[first_down_index]
# #                     second_up_base = input_seq2[second_up_index]

# #                     # if the next pair of base completely match
# #                     if first_down_base == complement_dict[second_up_base]:
# #                         first_subseq = first_subseq + first_down_base
# #                         second_subseq = second_up_base + second_subseq

# #                         extended_sub_seq_pairs.append((first_subseq, first_index,
# #                                                        first_down_index + 1, second_subseq,
# #                                                        second_up_index, second_index + self.seed_len,
# #                                                        mismatch_score))
# #                         first_down_index += 1
# #                         second_up_index -= 1
                        
# #                     else:
# #                         mismatch_score += 2
# #                         if mismatch_score <= self.allowed_max_mismatch: 
# #                             first_subseq = first_subseq + first_down_base
# #                             second_subseq = second_up_base + second_subseq
                            
# #                             first_down_index += 1
# #                             second_up_index -= 1
                            
# #                         else:
# #                             break
# #                 ### append the latest subseq pairs to the all_possible_subseqs_dict
# #                 all_possible_subseqs_dict[pair_index] = extended_sub_seq_pairs[-1]

# #                 ## for the itnernal intron, make sure the extension won't overpass the other subseq
# #             else:
# #                 # inward extension
# #                 first_down_index += 1
# #                 second_up_index -= 1
                
# #                 while (second_up_index - first_down_index) > 2: # maybe >= 2

# #                     first_down_base = input_seq1[first_down_index]
# #                     second_up_base = input_seq2[second_up_index]

# #                     # if the next pair of base completely match
# #                     if self.base_conversion(first_down_base) + self.base_conversion(second_up_base) == 0:
# #                         first_subseq = first_subseq + first_down_base
# #                         second_subseq = second_up_base + second_subseq
# #                         extended_sub_seq_pairs.append((first_subseq, first_index,
# #                                                        first_down_index + 1, second_subseq,
# #                                                        second_up_index, second_index + self.seed_len,
# #                                                        mismatch_score))
                        
# #                         first_down_index += 1
# #                         second_up_index -= 1
# #                     else:
# #                         mismatch_score += 2
# #                         if mismatch_score <= self.allowed_max_mismatch: 
# #                             first_subseq = first_subseq + first_down_base
# #                             second_subseq = second_up_base + second_subseq
                            
# #                             first_down_index += 1
# #                             second_up_index -= 1
                            
# #                         else:
# #                             break

# #                 # collect the latest extended subseqs
# #                 all_possible_subseqs_dict[pair_index] = extended_sub_seq_pairs[-1]

# #         self.all_possible_subseqs_dict = all_possible_subseqs_dict
    
    
# #     def interval_intersection_check(self, interval_pair_1, interval_pair_2):
# #         '''
# #             This function is used to check if one of the interval pair is contained in the other
# #         '''
# #         # example of one comb: ((0, (3, 22), (33932, 33951)), (1, (3, 16), (40174, 40187)))
# #         # first 0 is the list index from all_possible_subseqs_dict's pair index
# #         # followed by the start, end of the first arm in the first pair
# #         # followed by the start, end of the second arm in the first pair
# #         # followed by the information of the second pair
        
# #         # check if the first pair contains the second pair

# #         # first arm comparison
# #         first_arm_pair1 = (interval_pair_1[1][0] <= interval_pair_2[1][0]) and (interval_pair_1[1][1] >= interval_pair_2[1][1])
        
# #         # check if the second pair contains the first pair
# #         # first arm comparison
# #         first_arm_pair2 = (interval_pair_2[1][0] <= interval_pair_1[1][0]) and (interval_pair_2[1][1] >= interval_pair_1[1][1])


# #         # check if the first pair contains the second pair 
# #         if first_arm_pair1:

# #             # second arm comparison
# #             second_arm_pair1 = (interval_pair_1[2][0] <= interval_pair_2[2][0]) and (interval_pair_1[2][1] >= interval_pair_2[2][1])

# #             if second_arm_pair1:
# #                 # return the index for the pair needs to be dropped
# #                 return interval_pair_2[0]
# #             else:
# #                 return -1

# #         # check if the second pair contains the first pair
# #         elif first_arm_pair2:

# #             # second arm comparison
# #             second_arm_pair2 = (interval_pair_2[2][0] <= interval_pair_1[2][0]) and (interval_pair_2[2][1] >= interval_pair_1[2][1])

# #             if second_arm_pair2:
# #                 # return the index for the droped pair
# #                 return interval_pair_1[0]
# #             else:
# #                 return -1

# #         else:
# #             return -1
        
        
# #     def subseq_consolidation(self):
    
# #         '''
# #             This function takes in the result of the extended subseqs and remove all
# #             the redundant subseq pairs with the criteria: if a pair of subseqs completely 
# #             contained in another pair of subseqs, the contained pair of subseqs will be removed;
# #         '''
        
# #         self.subseq_extension()
        
# #         interval_pairs = []
        
# #         for key, value in self.all_possible_subseqs_dict.items():
# #             interval_pairs.append((key, value[1:3], value[4:6]))

# #         # obtain the all possible pair of combinations of the interval pairs

# #         interval_pairs_comb = combinations(interval_pairs, 2)
        
# #         # example of one comb: ((0, (3, 22), (33932, 33951)), (1, (3, 16), (40174, 40187)))
# #         # first 0 is the list index from all_possible_subseqs_dict's pair index
# #         # followed by the start, end of the first arm in the first pair
# #         # followed by the start, end of the second arm in the first pair
# #         # followed by the information of the second pair

# #         # check if one interval pair contains the other interval pair
# #         dropped_pair_index = set()
        
# #         for interval_pairs in interval_pairs_comb:
# #             dropped_pair_index.add(self.interval_intersection_check(*interval_pairs))

# #         # remove the index -1: that are either disjoint or intersected in either arms    
# #         if -1 in dropped_pair_index:
# #             dropped_pair_index.remove(-1)

# #         # drop the redundant keys from the dictionary
# #         # put the subseq results in the new dictionary

# #         selected_subseqs = {}

# #         selected_keys = set(self.all_possible_subseqs_dict.keys()).difference(dropped_pair_index)

# #         for key in selected_keys:
# #             selected_subseqs[key] = self.all_possible_subseqs_dict[key]
            
        
# #         # process the consolidated subseq dictionary
        
# #         # store the consolidated seq length
# #         consolidated_RCS_len = []
        
# #         if selected_subseqs:
# #             for item in selected_subseqs.values():
# #                 consolidated_RCS_len.append(len(item[0]))
# #         # assign np.nan to the seed_len that have no corresponding consolidated seqs
# #         else:
# #             consolidated_RCS_len.append(0)
            
# #         ## extract the distribution for the extended subseq pair from the selected_subseqs dict
        
# #         ## get the upper and lower subseq position pairs
# #         upper_lower_subseq_pos_pairs = []
# #         for subseq_pair in selected_subseqs.values():
# #             # subseq_pair[1]: start pos for upper subseq 
# #             # subseq_pair[4]: start pos for lower subseq
# #             # subseq_pair[2]: ending pos for upper subseq
# #             subseq_pair_len = int(subseq_pair[2]) - int(subseq_pair[1])
# #             upper_lower_subseq_pos_pairs.append((subseq_pair[1], subseq_pair[4], subseq_pair_len))
        
# #         ### calculate the total number of subseq  in each combination of intervals
# #         ### and store them in num_subseq_cross_intervals list
        
# #         num_subseq_cross_intervals = []
        
# #         for interval_comb in self.interval_combinations:
# #             upper_interval_pos = interval_comb[0]
# #             lower_interval_pos = interval_comb[1]
# #             ## check if the upper in upper_interval_pos and lower in lower_interval_pos
# #             ## upper or lower + 0.5 * subseq_pair_len length was used as the center of subseq pair
# #             num_subseq_cross_intervals.append(sum([(upper + 0.5 * subseq_len >= upper_interval_pos[0] and\
# #                                                       upper + 0.5 * subseq_len < upper_interval_pos[1]) and\
# #                                                      (lower + 0.5 * subseq_len >= lower_interval_pos[0] and\
# #                                                       lower + 0.5 * subseq_len < lower_interval_pos[1])\
# #                                                      for upper, lower, subseq_len in upper_lower_subseq_pos_pairs]))
            
# #         joint_subseq_pair_dist = list(np.array(num_subseq_cross_intervals))
# #         self.joint_subseq_pair_dist = joint_subseq_pair_dist
            
# #         self.rcm_summary_stats = list(np.percentile(consolidated_RCS_len, [0, 25, 50, 75, 100]))

# #         return self.key,self.num_rcm_kmers, self.num_rcm_kmers_per_10000_comp, self.joint_rcm_kmer_dist, self.rcm_summary_stats, self.joint_subseq_pair_dist


In [4]:
@ray.remote
class RCS_finder():
    '''
        This is a python class that can find the imperfect reverse complementary sequences (RCS) within
        a intronic sequence ranging from 50bps to 50kbps; it can also find the imperfect RCS of two
        flanking introns each ranging from 50bps to 50kbps.

        input_seq1: the Up flanking intron;
        input_seq2: the Lower flanking intron;
        is_flanking_introns: boolean value indicating whether dealing with two flanking introns
        if is_flanking_introns == False, the input_seq2 will be ignored and only input_seq1 will be used;
        seed_len: seed length for the initial RCS search
        seq_fraction_of_spacer: the minimal portion of the seq length between the two paired seed region
                                when dealing with one intron (between 0 to 1); set spacer to 0 if
                                spacer length is not used for filtering;
        allowed_seed_mismatch: allowed the number of mismatch in the seed region: default is 1 (i.e., 2 score)
    '''

    def __init__(self, key=None, input_seq1=None, input_seq2=None, is_flanking_introns=None, 
                 is_upper_intron=None, seq_fraction_of_spacer=None,
                 seed_len=None, allowed_seed_mismatch=None):
        
        ### input_seq1 is the upper intronic sequence
        ### input_seq2 is the lower intronic sequence
        self.key = key
        self.input_seq1 = input_seq1
        self.input_seq2 = input_seq2
        self.is_flanking_introns = is_flanking_introns
        self.is_upper_intron = is_upper_intron
        self.seq_fraction_of_spacer = seq_fraction_of_spacer
        self.seed_len = seed_len
        self.allowed_seed_mismatch = allowed_seed_mismatch
#         self.num_rcm_kmers = None
#         self.num_rcm_kmers_per_10000_comp = None


    def base_conversion(self, seq):
        '''
            convert the input sequence into the numerical representation
        '''
        base_mapping = {'A': 1, 'T': -1, 'C': 1j, 'G': -1j, 'N': 0}
        if len(seq) >= 2:
            seq_map = np.array([base_mapping[i] for i in seq])
        else:
            seq_map = base_mapping[seq]
        return seq_map

    def get_sub_seq_score(self, seq):

        '''
            This function starts with a input sequence and return
            the cummulative sum of the subsequence of length min_seq
        '''
        # get the augumented input sequence
        aug_seq = 'N' + seq
        # convert the augumented sequence into the numerical representation
        aug_seq_M = self.base_conversion(aug_seq)
        # get the cumulative sum of the augumented sequence
        aug_seq_cum_M = aug_seq_M.cumsum()
        # get the cumulative sum of the original input sequence
        seq_cum_M = aug_seq_cum_M[1:]
        # get subsequence score with length=self.seed_len
        sub_seq_score = seq_cum_M[self.seed_len - 1:] - aug_seq_cum_M[:len(aug_seq) - self.seed_len]
        return sub_seq_score

    def get_allowed_window_index(self):
        '''
            This function takes in the subseq_scores, and a fraction number (i.e., betwen 0 and 1) and return
            the allowed pair of window index that can be used for further subseqs verification
        '''
        
        if self.is_flanking_introns:
#             print(self.input_seq1)
#             print(self.input_seq2)
            # deal with flanking introns
            subseq_score1 = self.get_sub_seq_score(self.input_seq1).astype(np.complex64)
            subseq_score2 = self.get_sub_seq_score(self.input_seq2).astype(np.complex64)
            
            # put all possible pair-wise subseq addition score in the matrix
            # first subseq as the column, second subseq as the row
            subseq_scores_sum = subseq_score1.reshape((-1, 1)) + subseq_score2.reshape((1, -1))
            
            # get the sum of the absolute value of imaginary part and real part
            subseq_scores_abs_sum_mat = abs(subseq_scores_sum.imag) + abs(subseq_scores_sum.real)
            
            # get the index for the pair of subseqs that potentially have at most 1 mismatch (i.e., abs sum <= 2)
            first_window, second_window = np.where(subseq_scores_abs_sum_mat <= self.allowed_seed_mismatch)

            return first_window, second_window
        
        elif self.is_upper_intron:
            seq = self.input_seq1
        else:
            seq = self.input_seq2
        
#         print(seq)
        
        # deal with one intron only, use the seq
        subseq_scores = self.get_sub_seq_score(seq)

        ### this step is fast
        subseq_scores = subseq_scores.astype(np.complex64)

        #### this step takes about 7 seconds
        # put all possible pair-wise subseq addition score in the matrix
        subseq_scores_sum = subseq_scores.reshape((-1, 1)) + subseq_scores.reshape((1, -1))

        #### this step takes about 11 seconds
        # get the sum of the absolute value of imaginary part and real part
        subseq_scores_abs_sum_mat = abs(subseq_scores_sum.imag) + abs(subseq_scores_sum.real)

        #### this step takes about 9 seconds
        # obtain the lower triangular index including the diagnoal
        tril_index = np.tril_indices(subseq_scores_abs_sum_mat.shape[0])

        #### this step takes about 7 seconds
        # change the lower triangular value to be bigger than 2, thus don't need to filter due to the  matrix symetry
        subseq_scores_abs_sum_mat[tril_index] = self.allowed_seed_mismatch + 1

        #### this step takes about 9 seconds
        # get the index for the pair of subseqs that potentially have at most 1 mismatch (i.e., abs sum <= 2)
        first_window, second_window = np.where(subseq_scores_abs_sum_mat <= self.allowed_seed_mismatch)
        
         ### this step takes about 0.9 seconds
        # get the window distance for each pair of subseqs
        window_num_diff = second_window - first_window

        #### this step takes about 0.7 seconds
        # get the base pair distance for each pair of subseqs # 10 is the default arm length
        spacer = window_num_diff - self.seed_len

        ### this step takes about 0.3 seconds; set fraction_of_spacer at least to 0 to
        ### avoid the overlapping windows
        # base pair distance for each pair of subseqs at least half the total seq length
        allowed_index = (spacer >= len(seq) * self.seq_fraction_of_spacer)

        ### these two steps take about 0.6 seconds
        # get the allowed window index
        allowed_first_window = first_window[allowed_index]
        allowed_second_window = second_window[allowed_index]

        return allowed_first_window, allowed_second_window


    def subseq_validity_check(self):
        '''
            This function is used to check wether the subseq pairs identified in get_allowed_window_index function
            is valid or not by calling the check_valid_subseq_pairs function,
            This function also return the relative location of the valid rcm kmers as well as number of
            kmers per 10000 comparison
        '''
        first_window, second_window = self.get_allowed_window_index()

        valid_subseq_pairs_list = []

        complement_dict = {'A': 'T', 'G': 'C', 'T': 'A', 'C': 'G', 'N': 'N'}

        ### check to see if dealing with one introns or flanking introns
        if self.is_flanking_introns:
            input_seq1 = self.input_seq1
            input_seq2 = self.input_seq2
            
        elif self.is_upper_intron:
            input_seq1 = self.input_seq1
            input_seq2 = self.input_seq1
        else:
            input_seq1 = self.input_seq2
            input_seq2 = self.input_seq2

        for first_index, second_index in zip(first_window, second_window):

            # set it for each window pair
            seed_mismatch_score = 0

            first_subseq, second_subseq = input_seq1[first_index:self.seed_len + first_index], \
                                          input_seq2[second_index:self.seed_len + second_index]

            # check for the two ends first to make sure end is matching
            if first_subseq[0] == complement_dict[second_subseq[-1]]:
                # check the interior subseq
                for i in range(1, self.seed_len):
                    if first_subseq[i] != complement_dict[second_subseq[-i - 1]]:
                        seed_mismatch_score += 2
                        # if total mismatch already bigger than 2 abort the further checking
                        if seed_mismatch_score > self.allowed_seed_mismatch:
                            break

                if seed_mismatch_score <= self.allowed_seed_mismatch:
                    valid_subseq_pairs_list.append((first_subseq, first_index, second_subseq, \
                                                    second_index, seed_mismatch_score))
                    
#         print(valid_subseq_pairs_list)  
                ### The following code: extract the rcm kmer location distribution relative
        ### to the corresponding upper or lower intron

        ### create interval list for upper and lower introns based on the upper and lower intron length

        upper_equal_space_list = np.linspace(start=0, stop=len(input_seq1), num=5+1)

        lower_equal_space_list = np.linspace(start=0, stop=len(input_seq2), num=5+1)

        upper_interval_list = []
        lower_interval_list = []

        for i in range(len(upper_equal_space_list) - 1):
            upper_interval = (upper_equal_space_list[i], upper_equal_space_list[i + 1])
            upper_interval_list.append(upper_interval)

        for i in range(len(lower_equal_space_list) - 1):
            lower_interval = (lower_equal_space_list[i], lower_equal_space_list[i + 1])
            lower_interval_list.append(lower_interval)

        ### get the combination of interval from upper and lower introns 5 x 5 interval in this case
        ### if num = 6
        interval_combinations = list(product(upper_interval_list, lower_interval_list))
        
        ## save this interval_combinations to self.interval_combinations for the distribution of extended subseq pairs
        self.interval_combinations = interval_combinations
        

        ### get the upper and lower rcm kmer position pairs
        upper_lower_rcm_kmer_pos_pairs = []
        for i in valid_subseq_pairs_list:
            upper_lower_rcm_kmer_pos_pairs.append((i[1], i[3]))

        ### calculate the total number of rcm kmer in each combination of intervals
        ### and store them in num_rcm_kmer_cross_intervals list
        num_rcm_kmer_cross_intervals = []

        for interval_comb in self.interval_combinations:
            upper_interval_pos = interval_comb[0]
            lower_interval_pos = interval_comb[1]
            ## check if the upper in upper_interval_pos and lower in lower_interval_pos
            ## upper or lower + 0.5 * self.seed length was used as the center of rcm kmer

            num_rcm_kmer_cross_intervals.append(sum([(upper + 0.5 * self.seed_len >= upper_interval_pos[0] and\
                                                      upper + 0.5 * self.seed_len < upper_interval_pos[1]) and\
                                                     (lower + 0.5 * self.seed_len >= lower_interval_pos[0] and\
                                                      lower + 0.5 * self.seed_len < lower_interval_pos[1])\
                                                     for upper, lower in upper_lower_rcm_kmer_pos_pairs]))
        # if sum(num_rcm_kmer_cross_intervals) == 0:
        #     joint_rcm_kmer_dist = np.array(num_rcm_kmer_cross_intervals)
        # else:
        #     joint_rcm_kmer_dist = np.array(num_rcm_kmer_cross_intervals) / sum(num_rcm_kmer_cross_intervals) * 100
        ### return the raw number of rcm in different part of the introns maybe better than the normalized version

        joint_rcm_kmer_dist = np.array(num_rcm_kmer_cross_intervals)

                    
#         self.valid_subseq_pairs_list = valid_subseq_pairs_list
        
#         print(self.valid_subseq_pairs_list)
        
#         self.num_rcm_kmers = len(valid_subseq_pairs_list)
        
        
#         if self.is_flanking_introns:

#             num_comparison = (len(input_seq1) - self.seed_len + 1) * (len(input_seq2) - self.seed_len + 1)
#         else:
#             num_comparison = 1 / 2 * (len(input_seq1) - self.seed_len + 1) * (len(input_seq2) - self.seed_len)

#         ## get the number of rcm_kmers distrubution per 10000 comparison

#         joint_rcm_kmer_dist_per_million = (joint_rcm_kmer_dist / num_comparison) * 1000000

        return self.key, list(joint_rcm_kmer_dist)

In [5]:
# seq1 = BS_LS_flanking_seq_dict[all_keys_sorted_flanking_introns[0]]['U_flanking_seq']
# seq2 = BS_LS_flanking_seq_dict[all_keys_sorted_flanking_introns[0]]['L_flanking_seq']

# rcs_test = RCS_finder(key=all_keys_sorted_flanking_introns[0], input_seq1=seq1, input_seq2=seq2, is_flanking_introns=True,
#                                               seed_len=20,is_upper_intron=False,
#                                               allowed_seed_mismatch=0,
#                                               seq_fraction_of_spacer=0)

# rcs_test.subseq_validity_check()

In [6]:
np.array([ 1, 52, 0, 0, 447, 0, 104, 15])

array([  1,  52,   0,   0, 447,   0, 104,  15])

In [7]:
def rcs_flanking_introns_ray_chunk(seq_list, seed_len):
    rcs_list = []
    for key, value in seq_list:
        seq1 = value['U_flanking_seq']
#         print(len(seq1))
        seq2 = value['L_flanking_seq']
#         print(len(seq2))
        rcs_list.append(RCS_finder.remote(key=key, input_seq1=seq1, input_seq2=seq2, 
                                              is_flanking_introns=True,
                                              seed_len=seed_len, is_upper_intron=False, 
                                              seq_fraction_of_spacer=0,
                                              allowed_seed_mismatch=0))

    results = ray.get([rcs.subseq_validity_check.remote() for rcs in rcs_list])
    return results


def loop_flanking_introns_ray_chunk(BS_LS_flanking_seq_dict, chunk_keys, rcm_dump_folder, seed_len, flanking_len):
    '''
        This function loops chunk_keys and call the rcs_flanking_introns_ray_chunk function
        in each loop and collect the results in a dict and dump them in rcm_dump_folder
    '''
    rcs_flanking_introns_results = []
    result_dict = defaultdict(list)

    for i in range(len(chunk_keys)):

        seq_key = chunk_keys[i]

        seq_list = [(key, BS_LS_flanking_seq_dict[key]) for key in seq_key]

        result = rcs_flanking_introns_ray_chunk(seq_list=seq_list, seed_len=seed_len)

        rcs_flanking_introns_results.append(result)

#         print(rcs_flanking_introns_results)

        
    flat_list = [item for chunk in rcs_flanking_introns_results for item in chunk]

    for item in flat_list:
        rcm_num = [float(i) for i in item[1]]
        result_dict[item[0]].append(rcm_num)
        
#         rcm_num_norm = [float(i) for i in item[2]]
#         result_dict[item[0]].append(rcm_num_norm)
        
        
    file_name = f"to_{i + 1}_rcm_flanking_{flanking_len}_bps_introns_{seed_len}mer.json"

    with open(os.path.join(rcm_dump_folder, file_name), 'w') as f:
        json.dump(result_dict, f)

    print(f'flanking_rcm {i + 1} finished')

    ray.shutdown()


In [8]:
def rcs_upper_introns_ray_chunk(seq_list, seed_len):
    rcs_list = []
    for key, value in seq_list:
        seq1 = value['U_flanking_seq']
        seq2 = value['L_flanking_seq']
        rcs_list.append(RCS_finder.remote(key=key, input_seq1=seq1, input_seq2=seq2, 
                                              is_flanking_introns=False,
                                              seed_len=seed_len, is_upper_intron=True,
                                              seq_fraction_of_spacer=0,
                                              allowed_seed_mismatch=0))

    results = ray.get([rcs.subseq_validity_check.remote() for rcs in rcs_list])
    return results



def loop_upper_introns_ray_chunk(BS_LS_flanking_seq_dict, chunk_keys, rcm_dump_folder, seed_len, flanking_len):
    '''
        This function loops chunk_keys and call the rcs_upper_introns_ray_chunk function
        in each loop and collect the results in a list
    '''
    rcs_upper_introns_results = []
    result_dict = defaultdict(list)
    
    
    for i in range(len(chunk_keys)):

        seq_key = chunk_keys[i]

        seq_list = [(key, BS_LS_flanking_seq_dict[key]) for key in seq_key]

        result = rcs_upper_introns_ray_chunk(seq_list=seq_list, seed_len=seed_len)

        rcs_upper_introns_results.append(result)

#         print(rcs_upper_introns_results)

        
    flat_list = [item for chunk in rcs_upper_introns_results for item in chunk]

    for item in flat_list:
        rcm_num = [float(i) for i in item[1]]
        result_dict[item[0]].append(rcm_num)

    file_name = f"to_{i + 1}_rcm_upper_{flanking_len}_bps_introns_{seed_len}mer.json"

    with open(os.path.join(rcm_dump_folder, file_name), 'w') as f:
        json.dump(result_dict, f)

    print(f'upper_rcm {i + 1} finished')

    ray.shutdown()

In [9]:
def rcs_lower_introns_ray_chunk(seq_list, seed_len):
    rcs_list = []
    for key, value in seq_list:
        seq1 = value['U_flanking_seq']
        seq2 = value['L_flanking_seq']
        rcs_list.append(RCS_finder.remote(key=key, input_seq1=seq1, input_seq2=seq2, 
                                              is_flanking_introns=False,
                                              seed_len=seed_len, is_upper_intron=False,
                                              seq_fraction_of_spacer=0,
                                              allowed_seed_mismatch=0))

    results = ray.get([rcs.subseq_validity_check.remote() for rcs in rcs_list])
    return results


def loop_lower_introns_ray_chunk(BS_LS_flanking_seq_dict, chunk_keys, rcm_dump_folder, seed_len, flanking_len):
    '''
        This function loops chunk_keys and call the rcs_lower_introns_ray_chunk function
        in each loop and collect the results in a list
    '''
    rcs_lower_introns_results = []
    result_dict = defaultdict(list)
    
    

    for i in range(len(chunk_keys)):

        seq_key = chunk_keys[i]

        seq_list = [(key, BS_LS_flanking_seq_dict[key]) for key in seq_key]

        result = rcs_lower_introns_ray_chunk(seq_list=seq_list, seed_len=seed_len)

        rcs_lower_introns_results.append(result)
#         print(rcs_lower_introns_results)


    flat_list = [item for chunk in rcs_lower_introns_results for item in chunk]
    
    
    for item in flat_list:
        
        rcm_num = [float(i) for i in item[1]]
        result_dict[item[0]].append(rcm_num)
        
#         rcm_num_norm = [float(i) for i in item[2]]
#         result_dict[item[0]].append(rcm_num_norm)

    file_name = f"to_{i + 1}_rcm_lower_{flanking_len}_bps_introns_{seed_len}mer.json"
    with open(os.path.join(rcm_dump_folder, file_name), 'w') as f:
        json.dump(result_dict, f)

    print(f'lower_rcm {i + 1} finished')

    ray.shutdown()

In [10]:
ray.shutdown()

In [16]:
rcm_folder = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/rcm_scores/'

BS_LS_flanking_seq_dict_list = []

for flanking_intron_len in [200, 300, 400, 2500]:
    
    with open(f'/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/BS_LS_intronic_flanking_seq_{flanking_intron_len}_bps.json') as f:
        BS_LS_flanking_seq_dict = json.load(f)
        BS_LS_flanking_seq_dict_list.append(BS_LS_flanking_seq_dict)


In [17]:
[len(i['chr21|40792631|40807499|-']['L_flanking_seq']) for i in BS_LS_flanking_seq_dict_list]

[200, 300, 400, 2500]

In [18]:
all_keys = [key for key in BS_LS_flanking_seq_dict_list[0].keys()]

chunk_keys_flanking_introns = []
for i in range(0, len(all_keys), 50):
    chunk_keys_flanking_introns.append(all_keys[i:i+50])
    
chunk_keys_within_introns = []
for i in range(0, len(all_keys), 50):
    chunk_keys_within_introns.append(all_keys[i:i+50])

In [19]:
seed_len_list = [5, 7, 9, 11, 13]
flanking_len_list = [200, 300, 400, 2500]

for index, value in enumerate(BS_LS_flanking_seq_dict_list):
    
    for seed_len in seed_len_list:
        print(f'get flanking rcm feature for {flanking_len_list[index]} bps intron with seed length {seed_len}')

        loop_flanking_introns_ray_chunk(BS_LS_flanking_seq_dict=value, chunk_keys=chunk_keys_flanking_introns,
                                        rcm_dump_folder=rcm_folder, seed_len=seed_len,
                                       flanking_len=flanking_len_list[index])
        # get rcm feature for upper introns:
        print(f'get upper rcm feature for {flanking_len_list[index]} bps intron with seed length {seed_len}')

        loop_upper_introns_ray_chunk(BS_LS_flanking_seq_dict=value, chunk_keys=chunk_keys_within_introns,
                                     rcm_dump_folder=rcm_folder, seed_len=seed_len,
                                     flanking_len=flanking_len_list[index])
    # #     ### get rcm feature for lower introns:
        print(f'get lower rcm feature for {flanking_len_list[index]} bps intron with seed length {seed_len}')

        loop_lower_introns_ray_chunk(BS_LS_flanking_seq_dict=value, chunk_keys=chunk_keys_within_introns,
                                     rcm_dump_folder=rcm_folder, seed_len=seed_len,
                                     flanking_len=flanking_len_list[index])

get flanking rcm feature for 200 bps intron with seed length 5
flanking_rcm 485 finished
get upper rcm feature for 200 bps intron with seed length 5
upper_rcm 485 finished
get lower rcm feature for 200 bps intron with seed length 5
lower_rcm 485 finished
get flanking rcm feature for 200 bps intron with seed length 7
flanking_rcm 485 finished
get upper rcm feature for 200 bps intron with seed length 7
upper_rcm 485 finished
get lower rcm feature for 200 bps intron with seed length 7
lower_rcm 485 finished
get flanking rcm feature for 200 bps intron with seed length 9
flanking_rcm 485 finished
get upper rcm feature for 200 bps intron with seed length 9
upper_rcm 485 finished
get lower rcm feature for 200 bps intron with seed length 9
lower_rcm 485 finished
get flanking rcm feature for 200 bps intron with seed length 11
flanking_rcm 485 finished
get upper rcm feature for 200 bps intron with seed length 11
upper_rcm 485 finished
get lower rcm feature for 200 bps intron with seed length 11
